In [1]:
# autoreload
%reload_ext autoreload
%autoreload 2

In [5]:
### imports
# general
import pandas as pd
import re
import googlemaps

# custom
import utils

In [36]:
### load data
fp = "data/Orlando_data.xlsx"
df = pd.read_excel(fp, sheet_name="all_data")
df = utils.process_df(df)

## Cleaning coordinates

In [37]:
# assign value for each row to coords, or if no coords, location
df['loc'] = df['cleaned_coords'].fillna(df['location'])
df['loc'].unique()

array(['14°41′17.4″ S 145°28′03.6″ E / 14°41′47.0″ S 145°27′02.9″ E',
       '17°28′40.51″S, 149°50′22.80″W', '25.02° N, 80.49° W',
       '26°19.62’N, 80°04.33’W', 'Palmyra Atoll', '17.05°N, 111.3°E',
       'Fiji, UK aquarium', 'M', '23°26′S 151°52′E', 'Fiji',
       'Island Point, Panama', "Mo'orea, French Polynesia",
       'Almirante Bay, Panama', '23 33’S, 151 54’E', 'Hawaii, various',
       '21.955°N, 120.766°E / 21.931°N, 120.745°E',
       'Oku Reef, Okinawa, Japan', '16°24′43.2″ N 119°54′12.2″ E',
       '23°27′34″S 151°55′45″E',
       '18°12′7′′N, 109°28′5′′E / 18°16′40′′N, 109°44′3′′E',
       'Spermonde Archipelago, Sulawesi, Indonesia',
       '21°26′6″ N, 157°47′12″ W', '20°54′19.80″N, 86°50′6.20″W',
       '7°19′13.2″N, 134°30′00.6″E', 'N 26°42.15′, E 127°52.667’',
       '17°28′46.60″S 149°49′8.27″W', 'Indo-Pacific', "151°55'E, 23°27'S",
       '21°26′08.9′′ N, 157°47′12.0′′ W', '24.56257° N, 81.40009° W',
       'Biscayne National Park, Florida Keys', 'Key West, Flo

In [38]:
coords_array = df['cleaned_coords'].dropna().unique()
coords_array

array(['14°41′17.4″ S 145°28′03.6″ E / 14°41′47.0″ S 145°27′02.9″ E',
       '17°28′40.51″S, 149°50′22.80″W', '25.02° N, 80.49° W',
       '26°19.62’N, 80°04.33’W', '17.05°N, 111.3°E', '23°26′S 151°52′E',
       '23 33’S, 151 54’E', '21.955°N, 120.766°E / 21.931°N, 120.745°E',
       '16°24′43.2″ N 119°54′12.2″ E', '23°27′34″S 151°55′45″E',
       '18°12′7′′N, 109°28′5′′E / 18°16′40′′N, 109°44′3′′E',
       '21°26′6″ N, 157°47′12″ W', '20°54′19.80″N, 86°50′6.20″W',
       '7°19′13.2″N, 134°30′00.6″E', 'N 26°42.15′, E 127°52.667’',
       '17°28′46.60″S 149°49′8.27″W', "151°55'E, 23°27'S",
       '21°26′08.9′′ N, 157°47′12.0′′ W', '24.56257° N, 81.40009° W'],
      dtype=object)

In [ ]:
# def dms_to_decimal(degrees, minutes=0, seconds=0, direction=''):
#     """Convert degrees, minutes, and seconds to decimal degrees."""
#     try:
#         degrees, minutes, seconds = float(degrees), float(minutes), float(seconds)
#         if not (0 <= minutes < 60) or not (0 <= seconds < 60):
#             raise ValueError("Invalid minutes or seconds range.")
#         decimal = degrees + minutes / 60 + seconds / 3600
#         if direction in ['S', 'W']:
#             decimal *= -1
#         return decimal
#     except Exception as e:
#         print(f"Error converting DMS: {degrees}° {minutes}′ {seconds}″ {direction} → {e}")
#         return None
        

# def standardize_coordinates(coord_string):
#     """Convert various coordinate formats into decimal degrees."""
#     coord_string = coord_string.replace("′′", '″')
#     parts = re.split(r'\s*/\s*', coord_string)  # Split at '/' if present
#     decimal_coords = []

#     lat_lng_pattern = re.compile(r'''
#         ([NSEW])?\s*  # optional leading direction
#         (\d+(?:\.\d+)?)\s*[° ]\s*  # degrees (mandatory)
#         (?:(\d+(?:\.\d+)?)\s*[′'’]\s*)?  # optional Minutes
#         (?:(\d+(?:\.\d+)?)\s*[″"]\s*)?  # optional Seconds
#         ([NSEW])?  # optional trailing direction
#     ''', re.VERBOSE)
#     for part in parts:
#         lat_lng = lat_lng_pattern.findall(part)
        
#         if len(lat_lng) != 2:
#             print(f"Invalid coordinate pair: {part}")
#             continue
        
#         for coord in lat_lng:
#             dir1, deg, mins, secs, dir2 = coord
#             if dir1 is None and dir2 is None:
#                 print(f"Could not determine direction in: {part}")
#                 continue
#             decimal = dms_to_decimal(deg, mins or 0, secs or 0, dir1 or dir2)
#             if "N" in coord or "S" in coord:
#                 lat = decimal
#             elif "E" in coord or "W" in coord:
#                 lng = decimal
#         decimal_coords.append((lat, lng))

#     if len(decimal_coords) == 0:
#         print(f"Failed to parse: {coord_string}")
#         return None
#     if len(decimal_coords) == 1:
#         return decimal_coords[0]
#     else:
#         avg_lat = np.mean([c[0] for c in decimal_coords if c[0] is not None])
#         avg_lng = np.mean([c[1] for c in decimal_coords if c[1] is not None])
#         return (avg_lat, avg_lng)

for coord in coords_array:
    try:
        lat, lon = utils.standardize_coordinates(coord)
        print(f"Original: {coord} → Decimal Degrees: ({lat}, {lon})")
    except ValueError as e:
        print(f"Error processing coordinate {coord}: {e}")
    except Exception as e:
        print(f"Unexpected error processing coordinate {coord}: {e}\n")

Original: 14°41′17.4″ S 145°28′03.6″ E / 14°41′47.0″ S 145°27′02.9″ E → Decimal Degrees: (-14.692277777777779, 145.45923611111112)
Original: 17°28′40.51″S, 149°50′22.80″W → Decimal Degrees: (-17.477919444444442, -149.8396666666667)
Original: 25.02° N, 80.49° W → Decimal Degrees: (25.02, -80.49)
Original: 26°19.62’N, 80°04.33’W → Decimal Degrees: (26.327, -80.07216666666666)
Original: 17.05°N, 111.3°E → Decimal Degrees: (17.05, 111.3)
Original: 23°26′S 151°52′E → Decimal Degrees: (-23.433333333333334, 151.86666666666667)
Original: 23 33’S, 151 54’E → Decimal Degrees: (-23.55, 151.9)
Original: 21.955°N, 120.766°E / 21.931°N, 120.745°E → Decimal Degrees: (21.942999999999998, 120.75550000000001)
Original: 16°24′43.2″ N 119°54′12.2″ E → Decimal Degrees: (16.412, 119.9033888888889)
Original: 23°27′34″S 151°55′45″E → Decimal Degrees: (-23.459444444444443, 151.92916666666665)
Original: 18°12′7′′N, 109°28′5′′E / 18°16′40′′N, 109°44′3′′E → Decimal Degrees: (18.23986111111111, 109.60111111111111)

In [121]:
float('03.5')

3.5

In [71]:
test_coords_array[1].replace("′′", '"', inplace=True)

TypeError: str.replace() takes no keyword arguments

In [66]:
import re

pattern = re.compile(r'(\d+)[° ]\s*(\d+\.\d+)[′\'’]?\s*([NSEW])')
pmatch = pattern.match(test_coords_array[0])
pmatch.groups()

('26', '19.62', 'N')

In [42]:
type(coord)

str

## Filling missing coordinates with Google Maps API

In [ ]:
# load in others
denisa_df = pd.read_excel("data/Data extraction_Denisa.xlsx", sheet_name="Data extraction")
ashtyn_df = pd.read_excel("data/DataExtraction_AI.xlsx", sheet_name="Data_Extraction")

# concatenate data
GOOGLE_MAPS_API_KEY = utils.read_yaml("api_keys.yaml")['google_maps_api']

gmaps_client = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

locs = pd.DataFrame(denisa_df.Location.unique(), columns=["Location"])

# comment this out to avoid too many api calls
locs[["Latitude", "Longitude"]] = locs["Location"].apply(lambda x: utils.get_coordinates_from_gmaps(x, gmaps_client))


# denisa_df.Location.unique(), ashtyn_df.Location.unique()

In [ ]:
# read locations.yaml
locs = utils.read_yaml("data/locations.yaml")

locs_df = pd.DataFrame([
    {
        "DOI": entry["DOI"],
        "Latitude": entry["coordinates"]["Latitude"],
        "Longitude": entry["coordinates"]["Longitude"],
        "Location": entry["location"]
    }
    for entry in locs
])

# drop nans
locs_df = locs_df.dropna()
# read a list of dictionaries into a single dictionary, with the keys being the location names
# locs = {k: v for d in locs for k, v in d.items()}

# save dictionary to csv
# locs = pd.DataFrame(locs)
# locs.to_csv("data/locations.csv")

## Visualising

In [ ]:
### write locs to yaml, with nice formatting
locs_list = []
for _, row in doi_locs.iterrows():
    locs_list.append({
        'coordinates': {
            'Latitude': row['Latitude'],
            'Longitude': row['Longitude']
        },
        'location': row['Location'],
        'DOI': row['DOI']
    })

utils.write_yaml(locs_list, "data/locations.yaml")

In [ ]:
### plot spatial disribution of studies
locations = read_yaml("data/locations.yaml")

# create figure
fig, ax = plt.subplots(1, 1, figsize=(10, 20), subplot_kw={'projection': ccrs.PlateCarree()}, dpi=300)
ax.set_extent([-180, 180, -40, 40], crs=ccrs.PlateCarree())

# add features
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN, alpha=0.3)
ax.add_feature(cfeature.COASTLINE, edgecolor='lightgray')
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='gray', alpha=0.2)

# add locations with colors
colors = sns.color_palette("hsv", len(locations))
for i, data in enumerate(locations):
    coords = data["coordinates"]
    ax.plot(coords["Longitude"], coords["Latitude"], 'o', markeredgecolor='darkgrey', markersize=5, color=colors[i], transform=ccrs.PlateCarree(), label=data["location"])

# add legend
ax.legend(loc='lower center', bbox_to_anchor=(0.5, -1.1), ncol=len(locations)//10, fontsize=6);